In [1]:
# dependencies
import pandas as pd
import os
import matplotlib

In [2]:
# load excel file
file_path = os.path.join('raw_data','2007-2017-PIT-HUD.xlsx')
# read the excel file
df  = pd.read_excel(file_path, sheet_name=None, ignore_index=True)

# concatenate all worksheets
cdf = pd.concat([df])

# display consolidated data
cdf.head()

,Year,CoC Number,CoC Name,Total Homeless,Sheltered Homeless,Unsheltered Homeless,Homeless Individuals,Sheltered Homeless Individuals,Unsheltered Homeless Individuals,Homeless People in Families,...,Unsheltered Parenting Youth (Under 25),Parenting Youth Under 18,Sheltered Parenting Youth Under 18,Unsheltered Parenting Youth Under 18,Parenting Youth Age 18-24,Sheltered Parenting Youth Age 18-24,Unsheltered Parenting Youth Age 18-24,Children of Parenting Youth,Sheltered Children of Parenting Youth,Unsheltered Children of Parenting Youth
0,2017,AK-500,Anchorage CoC,1128,973,155,848,693,155,280,...,0,0,0,0,12,12,0,21,21,0
1,2017,AK-501,Alaska Balance of State CoC,717,578,139,506,367,139,211,...,0,0,0,0,10,10,0,18,18,0
2,2017,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",1092,875,217,932,715,217,160,...,0,6,6,0,2,2,0,10,10,0
3,2017,AL-501,Mobile City & County/Baldwin County CoC,606,341,265,426,179,247,180,...,0,0,0,0,12,12,0,18,18,0
4,2017,AL-502,Florence/Northwest Alabama CoC,155,133,22,155,133,22,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# drop columns that have NaN
cdf.dropna(axis=1, how='any')

# Add State and County
cdf["State"] = cdf["CoC Number"].apply(lambda x: x.split('-')[0])
cdf["County"] = cdf["CoC Name"].apply(lambda x: x.split('CoC')[0])

cdf["County"] = cdf["County"].apply(lambda x: x.replace("County",''))
cdf["County"] = cdf["County"].apply(lambda x: x.replace("City",''))
cdf["County"] = cdf["County"].apply(lambda x: x.replace("&",''))
cdf["County"] = cdf["County"].apply(lambda x: x.replace("Continuum of Care",''))
cdf["County"] = cdf["County"].apply(lambda x: x.replace("Balance of State",''))
cdf["County"] = cdf["County"].apply(lambda x: x.replace("Metropolitan Denver Homeless Initiative",'Denver'))
cdf["County"] = cdf["County"].apply(lambda x: x.replace("Salt Lake",'Salt Lake City'))
cdf["County"] = cdf["County"].apply(lambda x: x.replace("District of Columbia",'Washington'))
cdf["County"] = cdf["County"].apply(lambda x: x.split('/')[0])
cdf["County"] = cdf["County"].apply(lambda x: x.strip())

cdf["CityState"] = cdf["County"] + ", " + cdf["State"]

# Select columns of Interest
cdf = cdf[[ 'Year','CityState', 'State', 'County', 'Total Homeless', 'Sheltered Homeless', 'Unsheltered Homeless',
       'Homeless Individuals', 'Sheltered Homeless Individuals', 'Unsheltered Homeless Individuals', 
       'Homeless People in Families', 'Sheltered Homeless People in Families', 'Unsheltered Homeless People in Families',
       'Chronically Homeless Individuals', 'Sheltered Chronically Homeless Individuals', 
       'Unsheltered Chronically Homeless Individuals' ]]

# set the index to Year
cdf = cdf.set_index(['Year'])

# Check record count before writing data into a csv file
cdf.count()

CityState                                       399
State                                           399
County                                          399
Total Homeless                                  399
Sheltered Homeless                              399
Unsheltered Homeless                            399
Homeless Individuals                            399
Sheltered Homeless Individuals                  399
Unsheltered Homeless Individuals                399
Homeless People in Families                     399
Sheltered Homeless People in Families           399
Unsheltered Homeless People in Families         399
Chronically Homeless Individuals                399
Sheltered Chronically Homeless Individuals      399
Unsheltered Chronically Homeless Individuals    399
dtype: int64

In [4]:
cdf.to_csv("HUD_Consol_Data.csv")

In [8]:
# Filter data based on Cities of Interest
Cities = ["New York", "Philadelphia", "Boston", "Washington", "Chicago", "Minneapolis",
        "Denver", "Salt Lake City", "Seattle", "Los Angeles", "San Francisco", "Miami", 
        "Charlotte", "Atlanta", "Detroit", "Anchorage", "Honolulu", "Indianapolis"]

hud_cities_data = cdf[cdf["County"].isin(Cities)]
hud_cities_data = hud_cities_data.reset_index()

hud_cities_data = hud_cities_data.sort_values(by=["CityState","Year"])

hud_cities_data = hud_cities_data[hud_cities_data.CityState != "Washington, WA"]

hud_cities_data = hud_cities_data[[ 'Year','CityState', 'Total Homeless', 'Sheltered Homeless', 'Unsheltered Homeless',
       'Homeless Individuals', 'Sheltered Homeless Individuals', 'Unsheltered Homeless Individuals', 
       'Homeless People in Families', 'Sheltered Homeless People in Families', 'Unsheltered Homeless People in Families',
       'Chronically Homeless Individuals', 'Sheltered Chronically Homeless Individuals', 
       'Unsheltered Chronically Homeless Individuals' ]]

hud_cities_data = hud_cities_data.set_index(['Year'])

hud_cities_data.head()

,CityState,Total Homeless,Sheltered Homeless,Unsheltered Homeless,Homeless Individuals,Sheltered Homeless Individuals,Unsheltered Homeless Individuals,Homeless People in Families,Sheltered Homeless People in Families,Unsheltered Homeless People in Families,Chronically Homeless Individuals,Sheltered Chronically Homeless Individuals,Unsheltered Chronically Homeless Individuals
Year,,,,,,,,,,,,,
2017,"Philadelphia, PA",5693,4737,956,3341,2389,952,2352,2348,4,877,403,474
2017,"Salt Lake City, UT",2047,1886,161,1348,1187,161,699,699,0,121,90,31
2017,"San Francisco, CA",6858,2505,4353,6257,1924,4333,601,581,20,2112,537,1575
2017,"Seattle, WA",11643,6158,5485,8810,3406,5404,2833,2752,81,2483,704,1779
2017,"Washington, DC",7473,6576,897,3583,2686,897,3890,3890,0,1470,932,538


In [9]:
hud_cities_data.to_csv("HUD_Cities_Data.csv")